In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os
from sklearn.metrics import mean_squared_error
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import cPickle
import gzip
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path = '/home/chocolate/LSTM-source/Main_product_added_features/'
seq_new_15 = []
data = pd.read_csv(path + 'IF_main_cleaned_addfeature.csv')
# data
# data_new = data.drop(data.columns[11:],axis = 1)
# data_new.fillna('ffill')
ser_1 = data['Dis_EMA5_26'][1:]
ser_1.index = range(len(ser_1))
data['Dis_EMA5_26_PRED'] = ser_1
data_new = data.ix[:, 1:]
# 最后一分钟的预测特征为空值，会影响结果，予以删除
data_new = data_new[:-1]
data_new.index = range(len(data_new))

In [3]:
data_new['Last.Buy1price'][data_new['Last.Buy1price'] <= 2000] = None
data_new['Last.Buy1quantity'][data_new['Last.Buy1quantity'] >= 50] = None
data_new['Last.Sell1quantity'][data_new['Last.Sell1quantity'] >= 50] = None
data_new = data_new.dropna()
data_new.index = range(len(data_new))
data_new

,Latestprice,MaxPrice,MinPrice,Stockup,Volume,First.Latestprice,Last.Buy1price,Last.Buy1quantity,Last.Sell1price,Last.Sell1quantity,...,RaiseDown_2,RaiseDown_3,RaiseDown_4,RaiseDown_5,MA_5_PRED,MA_12_PRED,MA_26_PRED,boll_up_PRED,boll_down_PRED,Dis_EMA5_26_PRED
0,2153.6,2153.8,2153.2,-5,641,2153.6,2153.2000,5,2153.6,4,...,0.000371,0.007054,-0.006789,0.000247,0.00,0.000000,0.000000,0.000000,0.000000,0.032308
1,2154.0,2154.8,2153.4,272,1676,2154.0,2154.0000,2,2154.2,3,...,0.007054,-0.006789,0.000247,0.006896,0.00,0.000000,0.000000,0.000000,0.000000,0.199154
2,2170.0,2170.6,2169.6,-3,2795,2169.6,2170.0000,5,2171.2,1,...,0.000247,0.006896,-0.000154,0.000215,2157.72,0.000000,0.000000,0.000000,0.000000,0.900706
3,2155.6,2156.0,2154.4,502,1994,2155.6,2155.4000,5,2155.6,15,...,0.006896,-0.000154,0.000215,0.000031,2158.24,0.000000,0.000000,0.000000,0.000000,0.271578
4,2170.6,2171.0,2170.6,2,1602,2170.6,2170.4000,2,2171.2,1,...,0.000215,0.000031,-0.011516,0.004008,2164.48,0.000000,0.000000,0.000000,0.000000,3.470331
5,2170.0,2171.2,2170.0,6,1834,2170.6,2170.0000,5,2170.8,1,...,0.000031,-0.011516,0.004008,-0.004115,2164.68,0.000000,0.000000,0.000000,0.000000,4.111449
6,2171.0,2171.6,2170.0,-2,1716,2170.0,2170.6000,3,2171.6,4,...,-0.011516,0.004008,-0.004115,0.001554,2167.68,0.000000,0.000000,0.000000,0.000000,4.251916
7,2170.6,2171.6,2170.6,2,1196,2171.0,2170.4000,3,2171.2,3,...,0.004008,-0.004115,0.001554,0.011819,2165.56,0.000000,0.000000,0.000000,0.000000,-1.005488
8,2145.6,2147.0,2145.2,19,2262,2146.0,2145.2000,3,2145.8,7,...,-0.004115,0.001554,0.011819,-0.006837,2162.28,2160.566667,0.000000,2160.566667,2160.566667,-2.251632
9,2145.8,2145.8,2145.4,5,871,2145.4,2145.6000,4,2146.0,8,...,0.011819,-0.006837,0.000123,0.007161,2153.36,2159.633333,0.000000,0.000000,0.000000,-5.020417


In [4]:
data_new1 = data_new.drop(data_new.columns[10:],axis = 1)
#   数据归一化
data_array_per = np.array(data_new1)
min_max_scaler = preprocessing.MinMaxScaler()
data_array_per = min_max_scaler.fit_transform(data_array_per)
#  组时间序列
for k in range(len(data_array_per)):
    if k < len(data_array_per) and k >= 15:
        seq_new_15.append(data_array_per[k - 15:k])
seq_new_15 = np.array(seq_new_15)
print seq_new_15.shape

(140992, 15, 10)


In [8]:
# 通过查看均线之间的距离变化来标注股价的走势
def label_judge_EMADistance(data,time_length):
    longup_count = 0
    shortdown_count = 0
    longdown_count = 0
    shortup_count = 0
    is_longlast5 = False
    is_shortlast5 = False
    is_longbegin1 = False
    is_shortbegin1 = False    
    for i in range(len(data))[1:]:
        if data['Dis_EMA5_26_PRED'].iloc[i] > 0 and data['Dis_EMA5_26_PRED'].iloc[i] > data['Dis_EMA5_26_PRED'].iloc[i-1]:
            longup_count += 1
        if data['Dis_EMA5_26_PRED'].iloc[i] < 0 and data['Dis_EMA5_26_PRED'].iloc[i] < data['Dis_EMA5_26_PRED'].iloc[i-1]:
            shortdown_count += 1
        if data['Dis_EMA5_26_PRED'].iloc[i] > 0 and data['Dis_EMA5_26_PRED'].iloc[i] < data['Dis_EMA5_26_PRED'].iloc[i-1]:
            longdown_count += 1
        if data['Dis_EMA5_26_PRED'].iloc[i] < 0 and data['Dis_EMA5_26_PRED'].iloc[i] > data['Dis_EMA5_26_PRED'].iloc[i-1]:
            shortup_count += 1 
        if i == 1:
            if data['Dis_EMA5_26_PRED'].iloc[i] >= 1.5:
                is_longbegin1 = True
            if data['Dis_EMA5_26_PRED'].iloc[i] <= -1.5:
                is_shortbegin1 = True
        if i == len(data)-1:
            if data['Dis_EMA5_26_PRED'].iloc[i] >= 1.5:
                is_longlast5 = True
            if data['Dis_EMA5_26_PRED'].iloc[i] <= -1.5:
                is_shortlast5 = True
    if longup_count == time_length-1 and is_longlast5 == True:
        return [1,0,0,0,0]                            #  未来五分钟上涨
    if longdown_count == time_length-1 and is_longbegin1 == True:
        return [0,1,0,0,0]                            #  未来五分钟上涨趋势回调
    if shortdown_count == time_length-1 and is_shortlast5 == True:
        return [0,0,0,1,0]                            #  未来五分钟下跌
    if shortup_count == time_length-1 and is_shortbegin1 == True:
        return [0,0,0,0,1]                            #  未来五分钟下跌趋势回调
    
    return [0,0,1,0,0]                                #  未来五分钟平稳

In [9]:
seq_label = []
for i in range(len(data_new))[15:]:
    seq_label.append(label_judge_EMADistance(data_new[i:i+5],5))
seq_label_array = np.array(seq_label)
seq_label_array.shape

(140992, 5)

In [10]:
print u'上   涨：',seq_label.count([1,0,0,0,0]),seq_label.count([1,0,0,0,0]) /140992.0
print u'上涨回调：',seq_label.count([0,1,0,0,0]),seq_label.count([0,1,0,0,0]) /140992.0
print u'平稳震荡：',seq_label.count([0,0,1,0,0]),seq_label.count([0,0,1,0,0]) /140992.0
print u'下   跌：',seq_label.count([0,0,0,1,0]),seq_label.count([0,0,0,1,0]) /140992.0
print u'下跌回调：',seq_label.count([0,0,0,0,1]),seq_label.count([0,0,0,0,1]) /140992.0

上   涨： 3031 0.0214976736269
上涨回调： 951 0.0067450635497
平稳震荡： 133585 0.947465104403
下   跌： 2747 0.0194833749433
下跌回调： 678 0.00480878347708


In [11]:
from random import shuffle
zip_seq = zip(seq_new_15,seq_label)
shuffle(zip_seq)
seq_new_15,seq_label = zip(*zip_seq)
seq_new_15 = np.array(seq_new_15) 
seq_label = np.array(seq_label)
print seq_new_15.shape, seq_label.shape

(140992, 15, 10) (140992, 5)


In [12]:
train_data = seq_new_15[:100000]
train_label = seq_label[:100000]

test_data = seq_new_15[100000:]
test_label = seq_label[100000:]
train_data.shape, test_label.shape

((100000, 15, 10), (40992, 5))

In [13]:
path_new = '/home/chocolate/LSTM-source/train_test_data/'
f1 = file(path_new + 'train_data_7.npy','wb')
np.save(f1,train_data)
f1.close()
f2 = file(path_new + 'test_data_7.npy','wb')
np.save(f2,test_data)
f2.close()
f3 = file(path_new + 'train_label_7.npy','wb')
np.save(f3,train_label)
f3.close()
f4 = file(path_new + 'test_label_7.npy','wb')
np.save(f4,test_label)
f4.close()

In [14]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import cPickle
import gzip
from sklearn import preprocessing
%matplotlib inline

In [15]:
path_new = '/home/chocolate/LSTM-source/train_test_data/'
# 读取训练集和测试集
f1 = file(path_new + 'train_data_7.npy','rb')
train_data = np.load(f1)
f2 = file(path_new + 'test_data_7.npy','rb')
test_data = np.load(f2)
f3 = file(path_new + 'train_label_7.npy','rb')
train_label = np.load(f3)
f4 = file(path_new + 'test_label_7.npy','rb')
test_label = np.load(f4)

In [17]:
import random

learning_rate = 0.001 # 学习速率
training_iters = 500000  # 训练总步长
batch_size = 2000 # 每小段步长
display_step = 10  # 每隔100小段显示输出

# 参数设置
n_input = 10 # 特征数量
n_steps = 15 # 时间序列长度
n_hidden = 500 # 隐藏层神经元个数
n_classes = 5 # 分类数量

tf.reset_default_graph()  # 重置流图
# 设置输入输出格式大小
xtr = tf.placeholder("float", [None, n_steps, n_input])
ytr = tf.placeholder("float", [None,n_classes])

# 定义权重和偏移量
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# def pred():
    
def RNN(x, weights, biases):

    # 更改数据集的格式，使其满足RNN网络的输入需求
    # 现在输入格式: (batch_size, n_steps, n_input)
    # 需要的格式: 'n_steps' 个tensor，每个的格式为 (batch_size, n_input)

    # 变换batch_size 和 n_steps的位置
    x = tf.transpose(x, [1, 0, 2])
    # 重新调整格式 (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # 分割成一系列的 'n_steps' 个tensors，每个的格式为 (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # 定义LSTM神经网络结构
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # 获取输出和状态
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)
    # 返回输出
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
# 获取预测值
pred = RNN(xtr, weights, biases)
# train_label = tf.split(0, batch_size, train_label)
# 取下一小段的输入
def next_batch_tr(batch_size):
    step = random.randint(1, len(train_data)/batch_size)
    next_batch_data = train_data[(step-1)*batch_size:step*batch_size]
    next_batch_label = train_label[(step-1)*batch_size:step*batch_size]   
    return next_batch_data, next_batch_label



In [19]:
# 定义代价函数和最优化方法
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, ytr))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 评估模型准确率
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(ytr, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 变量初始化
init = tf.initialize_all_variables()
part_saver = tf.train.Saver()
# 创建会话
sess = tf.InteractiveSession()
sess.run(init)
step = 1
    # 一直迭代，直到最大步长
while step * batch_size < training_iters:
    batch_x, batch_y = next_batch_tr(batch_size)
    batch_x = batch_x.reshape((-1, n_steps, n_input))
        # 利用最优化方法调整权值
    sess.run(optimizer, feed_dict={xtr: batch_x, ytr: batch_y})
    if step % display_step == 0:
            # 计算准确率和损失函数
        acc = sess.run(accuracy, feed_dict={xtr: batch_x, ytr: batch_y})
        loss = sess.run(cost, feed_dict={xtr: batch_x, ytr: batch_y})
        # 每隔一定步长显示训练准确率和损失率
        print("Iter " + str(step * batch_size) + ", Minibatch Loss= " +
                "{:.6f}".format(loss) + ", Training Accuracy= " +
                "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")

saver_path = part_saver.save(sess, "/home/chocolate/LSTM-source/models/model_7/predict_EMA.ckpt")
print "Model saved." 


Iter 20000, Minibatch Loss= 0.514184, Training Accuracy= 0.94700
Iter 40000, Minibatch Loss= 0.243788, Training Accuracy= 0.95850
Iter 60000, Minibatch Loss= 0.271280, Training Accuracy= 0.95300
Iter 80000, Minibatch Loss= 0.321706, Training Accuracy= 0.93550
Iter 100000, Minibatch Loss= 0.268856, Training Accuracy= 0.94900
Iter 120000, Minibatch Loss= 0.268724, Training Accuracy= 0.94650
Iter 140000, Minibatch Loss= 0.259270, Training Accuracy= 0.94800
Iter 160000, Minibatch Loss= 0.306565, Training Accuracy= 0.93550
Iter 180000, Minibatch Loss= 0.265113, Training Accuracy= 0.94700
Iter 200000, Minibatch Loss= 0.260845, Training Accuracy= 0.94800
Iter 220000, Minibatch Loss= 0.236571, Training Accuracy= 0.95350
Iter 240000, Minibatch Loss= 0.274113, Training Accuracy= 0.94500
Iter 260000, Minibatch Loss= 0.277762, Training Accuracy= 0.94500
Iter 280000, Minibatch Loss= 0.235548, Training Accuracy= 0.95450
Iter 300000, Minibatch Loss= 0.237497, Training Accuracy= 0.95350
Iter 320000, M

In [ ]:
saver = tf.train.Saver()
sess = tf.InteractiveSession()
saver.restore(sess, "/home/chocolate/LSTM-source/models/model_7/predict_EMA.ckpt")
print "Model restored."
test_data1 = test_data.reshape((-1, n_steps, n_input))
# test_label1 = test_label[:test_len]
# print("Testing Accuracy:",
#         sess.run(accuracy, feed_dict={xtr: test_data1, ytr: test_label})
pred_1 = sess.run(pred, feed_dict={xtr: test_data1})
        # sess_1.close()

predict = 1 + pred_1.argmax()
print predict